# SQL Querying

In [56]:
import csv, sqlite3

In [62]:
conn = sqlite3.connect('transactions.db')
cur = conn.cursor()
cur.execute("CREATE TABLE transactions (clnt_id INTEGER, bank_id INTEGER, acc_id INTEGER, txn_id INTEGER, txn_date TEXT, desc TEXT, amt REAL, cat TEXT, merchant TEXT)")

In [63]:
with open('data.csv', 'r') as f:
    dr = csv.DictReader(f)
    to_db = [(i['clnt_id'], i['bank_id'], i['acc_id'], i['txn_id'], i['txn_date'], i['desc'], i['amt'], i['cat'], i['merchant']) for i in dr]

cur.executemany("INSERT INTO transactions (clnt_id, bank_id, acc_id, txn_id, txn_date, desc, amt, cat, merchant) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", to_db)
conn.commit()
conn.close()


In [64]:
def get_transactions_from_db(query):
    import pandas as pd
    import sqlite3
    
    # Connect to database
    conn = sqlite3.connect('transactions.db')
    
    try:
        # Execute query and return results as DataFrame
        df = pd.read_sql_query(query, conn)
        return df
    finally:
        # Ensure connection is closed
        conn.close()

In [71]:
get_transactions_from_db("SELECT * FROM transactions WHERE cat='Shops'")

,clnt_id,bank_id,acc_id,txn_id,txn_date,desc,amt,cat,merchant
0,6,1,1,54,31/07/2023 0:00,CLOC Advance,6.286,Shops,NA
1,6,1,1,27,31/07/2023 0:00,CLOC Advance,6.286,Shops,NA
2,6,1,1,11,01/08/2023 0:00,CLOC Advance,2.268,Shops,NA
3,28,1,1,108,25/07/2023 0:00,1INFINITELOOP@ 07/25 #68 PMNT RCVD 1INFINITELO...,59.100,Shops,1INFINITE
4,28,1,1,136,14/08/2023 0:00,1INFINITELOOP@ 08/14 #68 PMNT RCVD 1INFINITELO...,4.924,Shops,1INFINITE
...,...,...,...,...,...,...,...,...,...
7413,880,489,558,85911,03/07/2023 0:00,GOODYEAR ONLINE PMT 230630 3168 Maryse Hemant,-6.000,Shops,Goodyear
7414,880,489,558,174063,05/09/2023 0:00,PURCHASE AUTHORIZED ON 09/01 PlayStation Netwo...,-4.238,Shops,PlayStation Network
7415,880,487,556,70388,18/09/2023 0:00,PURCHASE AUTHORIZED ON 09/17 PlayStation Netwo...,-1.058,Shops,PlayStation Network
7416,880,487,556,125142,26/06/2023 0:00,RECURRING PAYMENT AUTHORIZED ON 06/25 PlayStat...,-2.542,Shops,PlayStation Network


---

# **Query**: What is the amount I have spent on Uber in the last 5 months? 

## 1. Get the required data

**Issues**:
1. 3866 unique merchants and 10000+ unique descriptions that cannot be fed into the model.

**Solution:**
1. Combining cleaned description and merchant and, vectorizing them.
2. Use similarity search with the query to get the initial subset of data needed.

In [73]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('transactions.db')
cur = conn.cursor()

In [84]:
query = "SELECT * FROM transactions WHERE merchant='Uber'"
data = pd.read_sql_query(query, conn)

In [85]:
data

,clnt_id,bank_id,acc_id,txn_id,txn_date,desc,amt,cat,merchant
0,880,380,427,60354,13/06/2023 0:00,Withdrawal MARSHALLS #1036 / OLD TROY PIKE HUB...,-20.204,Department Stores,Uber
1,507,1,1,97,22/06/2023 0:00,Zelle payment from ...,1.400,Travel,Uber
2,507,1,1,187,15/06/2023 0:00,Zelle payment from ...,1.400,Travel,Uber
3,507,1,2,193,03/07/2023 0:00,Zelle payment from ...,4.000,Travel,Uber
4,507,1,1,236,13/06/2023 0:00,Zelle payment from ...,1.400,Travel,Uber
...,...,...,...,...,...,...,...,...,...
580,880,851,982,158309,05/06/2023 0:00,FOREIGN EXCHANGE RATE ADJUSTMENT FEE 06/02UBER...,-0.058,Bank Fees,Uber
581,880,851,982,95126,02/06/2023 0:00,FOREIGN EXCHANGE RATE ADJUSTMENT FEE 06/01UBER...,-0.054,Bank Fees,Uber
582,880,851,982,48175,20/07/2023 0:00,FOREIGN EXCHANGE RATE ADJUSTMENT FEE 07/19UBER...,-0.066,Bank Fees,Uber
583,880,851,982,147904,07/08/2023 0:00,FOREIGN EXCHANGE RATE ADJUSTMENT FEE 08/04UBER...,-0.064,Bank Fees,Uber


## 2. Vectorizing `description` and `merchant`

In [232]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

df = pd.read_csv("data.csv")
model = SentenceTransformer('all-MiniLM-L6-v2') 

In [157]:
df.head()

,clnt_id,bank_id,acc_id,txn_id,txn_date,desc,amt,cat,merchant,desc_merchant_cat,cleaned_text
0,6,1,1,54,2023-07-31,CLOC Advance,6.286,Shops,,CLOC Advance Shops,cloc advance shops
1,6,1,1,27,2023-07-31,CLOC Advance,6.286,Shops,,CLOC Advance Shops,cloc advance shops
2,6,1,1,11,2023-08-01,CLOC Advance,2.268,Shops,,CLOC Advance Shops,cloc advance shops
3,28,1,1,108,2023-07-25,1INFINITELOOP@ 07/25 #68 PMNT RCVD 1INFINITELO...,59.100,Shops,1INFINITE,1INFINITELOOP@ 07/25 #68 PMNT RCVD 1INFINITELO...,infiniteloop pmnt rcvd infiniteloopapp ca i...
4,28,1,1,136,2023-08-14,1INFINITELOOP@ 08/14 #68 PMNT RCVD 1INFINITELO...,4.924,Shops,1INFINITE,1INFINITELOOP@ 08/14 #68 PMNT RCVD 1INFINITELO...,infiniteloop pmnt rcvd infiniteloopapp ca i...


In [233]:
from utils import clean_text
df['desc'] = df['desc'].fillna('')
df['merchant'] = df['merchant'].fillna('')
df['cat'] = df['cat'].fillna('')
df['txn_date'] = pd.to_datetime(df['txn_date'], dayfirst=True, errors='coerce')

df['desc_merchant_cat'] = df['desc'] + ' ' + df['merchant'] + ' ' + df['cat']

df['cleaned_text'] = df['desc_merchant_cat'].apply(clean_text)

unique_cleaned_text = df['cleaned_text'].unique()

In [147]:
# Vectorize description and merchant combined
embeddings = model.encode(unique_cleaned_text, show_progress_bar=True).astype('float32')

# Create a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])    
index.add(embeddings)

index_to_desc = {i: desc for i, desc in enumerate(unique_cleaned_text)}

Batches: 100%|██████████| 1761/1761 [06:16<00:00,  4.68it/s]


In [148]:
faiss.write_index(index, "index.faiss")
np.save("embeddings.npy", embeddings)

In [234]:
def transaction_retrieval(query):
    query_vec = model.encode([clean_text(query)]).astype('float32')
    D, I = index.search(query_vec, 200)

    # Preserve order by keeping the list
    ordered_indices = I[0]
    ordered_texts = [index_to_desc[i] for i in ordered_indices]

    # Create a DataFrame using the indices directly (assuming you have a mapping from FAISS index to df index)
    ordered_df = df[df['cleaned_text'].isin(ordered_texts)]
    ordered_df = ordered_df.set_index('cleaned_text').loc[ordered_texts].reset_index()

    return ordered_texts, ordered_df

In [235]:
texts, df = transaction_retrieval("uber ride")

In [236]:
texts

['uber san francisco ca  uber travel',
 'uber   pmnt rcvd uber san francisco ca ckcd  uber travel',
 'marathon  huber heights oh uber convenience stores',
 'uber san francisco ca  uber payroll',
 'purchase  uber eleven helpubercomca  uber travel',
 'uber instantpay san francisco ca  uber payroll',
 'uber   pmnt rcvd uber san francisco ca uber payroll',
 'withdrawal target t   executive blvd huber heights oh  uber supermarkets and groceries',
 'withdrawal debit chip  raising canes  huber heights oh date   uber restaurants',
 'uber   pmnt rcvd uber san francisco ca ckcd  uber payroll',
 'scooters coffee  pleasant hill ia  restaurants',
 'journeys kidz a journeys clothing and accessories',
 'visa direct card uber travel',
 'uber   pmnt rcvd uber visa direct wi uber travel',
 'journeys clothing and accessories',
 'uber                  visa direct  us uber payroll',
 'journeys kidz journeys clothing and accessories',
 'pos debit      journeys  brooklyn  us journeys clothing and accessories

---

## 3. Using vectors to search for the distinct values using `ChromaDB`

Using vector search to get the necessary set of distinct values in `merchant` column, sizing down the load to the LLM

In [237]:
import chromadb
import pandas as pd


In [262]:
dfc = pd.read_csv("data.csv")
dfc['merchant'] = dfc['merchant'].fillna('').apply(clean_text)
dfc['desc'] = dfc['desc'].fillna('').apply(clean_text)
dfc['merc_desc'] = dfc['merchant'] + ' ' + dfc['desc']

merc_desc = dfc['merc_desc'].unique()
merc_desc_embeddings = model.encode(merc_desc, show_progress_bar=True).astype('float32')

Batches: 100%|██████████| 1694/1694 [03:25<00:00,  8.26it/s]


In [263]:
np.save("merchant_description_embeddings.npy", merc_desc_embeddings)

In [265]:
def batch_add_to_chroma(collection, documents, embeddings, ids, batch_size=1000):
    for i in range(0, len(documents), batch_size):
        collection.add(
            documents=documents[i:i+batch_size],
            embeddings=embeddings[i:i+batch_size],
            ids=ids[i:i+batch_size]
        )

In [266]:
client = chromadb.Client()
merchant_description_collection = client.get_or_create_collection("merchant_description_embeddings")
batch_add_to_chroma(
    merchant_description_collection,
    merc_desc.tolist(),
    merc_desc_embeddings,
    [str(i) for i in range(len(merc_desc))]
)

In [267]:
def get_relevant_merchants(query, k=20):
    query_vec = model.encode([clean_text(query)]).tolist()

    result = merchant_description_collection.query(
        query_embeddings=query_vec,
        n_results=k,
        include=["documents", "distances"]
    )

    merchants = result['documents'][0]
    scores = result['distances'][0]
    return merchants, scores

In [255]:
# Group descriptions and map them to the most common merchant
desc_to_merchant = (
    df[~df['merchant'].isna()]
    .groupby('desc')['merchant']
    .agg(lambda x: x.mode().iloc[0])  # Most common merchant
    .to_dict()
)


In [268]:
def search_merchants_and_fill(query, top_k=10):
    query_vec = model.encode([clean_text(query)]).tolist()

    result = merchant_description_collection.query(
        query_embeddings=query_vec,
        n_results=top_k,
        include=["documents", "distances"]
    )

    merchants = result['documents'][0]
    distances = result['distances'][0]

    return merchants, distances

In [271]:
search_merchants_and_fill("cabs")

(['freddys freddys  cabot ar ',
  'mcdonalds mcdonalds f cabazon ca ',
  ' b cabs home franois ciara id ',
  ' highway pickers',
  'la cabana dbt crd la cabana mexican rest washington ga dswyvg card  ',
  ' nachos truck',
  ' hwy truck st',
  ' jack in the box  mesquite tx ',
  'stripes stripes ',
  'wendys pos deb    wendy s   railroad ave rifle        co c '],
 [0.9545683264732361,
  0.9833301305770874,
  1.0106351375579834,
  1.0263371467590332,
  1.0896679162979126,
  1.0976074934005737,
  1.1003589630126953,
  1.115720510482788,
  1.1243226528167725,
  1.1261675357818604])

---

## 3. Vectorizing separately to not mix up semantics

**Issue Previously:**
1. Vectorizing combined values when search gives up noises

**Solutions:**
1. Vectorize and perform search separately.
2. Combine both based on scores and give the results.

**Issue**: Not vectorizing catergory along with merchant causing significant loss in relevant search results. \
**Solution**: Combine `cat` and `merchant`

In [358]:
dfs = pd.read_csv("data.csv")

dfs['desc'] = dfs['desc'].fillna('')
dfs['merchant_cat'] = dfs['merchant'].fillna('') + ' ' + dfs['cat'].fillna('')

dfs['desc_clean'] = dfs['desc'].apply(clean_text)
dfs['merchant_cat_clean'] = dfs['merchant_cat'].apply(clean_text)

desc = dfs['desc_clean'].unique()
merchant_cat = dfs['merchant_cat_clean'].unique()

desc_embeddings = model.encode(desc, show_progress_bar=True).astype('float32')
merchant_cat_embeddings = model.encode(merchant_cat, show_progress_bar=True).astype('float32')

np.save("desc_embeddings.npy", desc_embeddings)
np.save("merchant_cat_embeddings.npy", merchant_cat_embeddings)

Batches: 100%|██████████| 131/131 [00:13<00:00,  9.67it/s]


In [360]:
client = chromadb.Client()
desc_collection = client.get_or_create_collection("desc_embeddings")
merchant_cat_collection = client.get_or_create_collection("merchant_cat_embeddings")


batch_add_to_chroma(
    desc_collection,
    desc.tolist(),
    desc_embeddings,
    [str(i) for i in range(len(desc))]
)

batch_add_to_chroma(
    merchant_cat_collection,
    merchant_cat.tolist(),
    merchant_cat_embeddings,
    [str(i) for i in range(len(merchant_cat))]
)

In [377]:
def search(dfs, query, top_k=20):
    query_vec = model.encode([clean_text(query)]).tolist()

    # Perform independent searches
    merc_cat_result = merchant_cat_collection.query(
        query_embeddings=query_vec, n_results=top_k, include=["documents"]
    )
    desc_result = desc_collection.query(
        query_embeddings=query_vec, n_results=top_k, include=["documents"]
    )

    # Extract unique results
    merc_cat_docs = set(merc_cat_result['documents'][0])
    desc_docs = set(desc_result['documents'][0])

    # Simple union of both sets
    # all_docs = list(merc_cat_docs.union(desc_docs))
    return dfs[dfs['merchant_cat_clean'].isin(merc_cat_docs) | dfs['desc_clean'].isin(desc_docs)]


In [327]:
def search(query, top_k=20, merchant_weight=0.5):
    query_vec = model.encode([clean_text(query)]).tolist()

    merc_result = merchant_collection.query(query_embeddings=query_vec, n_results=top_k, include=["documents", "distances"])
    desc_result = desc_collection.query(query_embeddings=query_vec, n_results=top_k, include=["documents", "distances"])
    
    m_hits = {doc: 1 - dist * merchant_weight for doc, dist in zip(merc_result['documents'][0], merc_result['distances'][0])}
    d_hits = {doc: 1 - dist * (1 - merchant_weight) for doc, dist in zip(desc_result['documents'][0], desc_result['distances'][0])}

    # Combine scores
    combined_scores = {}
    for doc in set(m_hits.keys()).union(d_hits.keys()):
        combined_scores[doc] = m_hits.get(doc, 0) + d_hits.get(doc, 0)

    # Sort and return top combined results
    sorted_combined = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_combined[:top_k]    

In [339]:
def get_matching_transactions(combined_hits):
    merchants = set()
    descs = set()
    for doc, _ in combined_hits:
        if doc in dfc['merchant'].values:
            merchants.add(doc)
        if doc in dfc['desc'].values:
            descs.add(doc)
    return dfc[dfc['merchant'].isin(merchants) | dfc['desc'].isin(descs)]

In [378]:
s_res = search(dfs[dfs['cat'] == "Travel"], query="How much did I spend on cabs?")


In [375]:
print("Unique merchants: ", s_res.merchant.unique())
# print("Unique descriptions: ", res.desc.unique())
print("Unique categories: ", s_res.cat.unique())


Unique merchants:  ['Uber' 'CREDIT']
Unique categories:  ['Travel']


---

## 4. Using `clnt_id` and `cat` as metadata to get the right subset

In [390]:
dfm = pd.read_csv("data.csv")

dfm = dfm.dropna(subset=['clnt_id', 'desc'])

dfm['desc_clean'] = dfm['desc'].apply(clean_text)

unique_desc_by_clnt = dfm.drop_duplicates(subset=['clnt_id', 'desc_clean'])


In [ ]:
client = chromadb.Client()
collection = client.get_or_create_collection("txn_collection")
docs = unique_desc_by_clnt['desc_clean'].tolist()
embeddings = model.encode(docs, show_progress_bar=True).astype('float32')
ids = [str(i) for i in range(len(docs))]
metadata = unique_desc_by_clnt[['clnt_id', 'cat']].to_dict(orient='records')

In [402]:
def batch_add_to_chroma_meta(collection, documents, embeddings, ids, metadatas, batch_size=1000):
    for i in range(0, len(documents), batch_size):
        collection.add(
            documents=documents[i:i+batch_size],
            embeddings=embeddings[i:i+batch_size],
            ids=ids[i:i+batch_size],
            metadatas=metadatas[i:i+batch_size]
        )

batch_add_to_chroma_meta(
    collection,
    docs,
    embeddings,
    ids,
    metadata
)

In [427]:
def search(df, query, client_id, category=None, top_k=100):
    where = {'$and': [{'clnt_id': {'$eq': client_id}},
    {'cat': {'$eq': category}}]}

    query_vec = model.encode([clean_text(query)]).tolist()
    result = collection.query(
        query_embeddings=query_vec,
        n_results=top_k,
        include=["documents"],
        where=where
    )

    docs = set(result['documents'][0])

    return df[(df['desc_clean'].isin(docs)) & (df['clnt_id'] == client_id) & (df['cat'] == category)]

In [428]:
res = search(dfm, "what are my spending on cabs?", 880, "Travel")
res

,clnt_id,bank_id,acc_id,txn_id,txn_date,desc,amt,cat,merchant,desc_clean
24604,880,899,1033,6708,17/07/2023 0:00,Green Dot Bank Transfer,-2.000,Travel,NaN,green dot bank transfer
24605,880,148,166,32792,03/07/2023 0:00,From Maryse Hemant 3168 ShareFunds Transfer vi...,1.000,Travel,NaN,from maryse hemant sharefunds transfer via mo...
24606,880,176,196,40975,05/09/2023 0:00,Visa Direct Card 6643uber,0.430,Travel,Uber,visa direct card uber
24612,880,204,230,66040,05/06/2023 0:00,Zelle payment to Maryse Hemant JPM999xj8lkd,-6.000,Travel,Zelle,zelle payment to maryse hemant jpmxjlkd
24614,880,607,699,48468,10/08/2023 19:00,Weirdo #Rr,-6.762,Travel,NaN,weirdo rr
...,...,...,...,...,...,...,...,...,...,...
24712,880,195,217,17163,27/09/2023 0:00,Cash App*Maryse Hemant*A,-40.000,Travel,CASH APP,cash appmaryse hemanta
24715,880,527,603,47711,05/09/2023 0:00,Maryse Hemant PAYMENT ID BBT191713026 ZELLE PA...,-17.000,Travel,Zelle,maryse hemant payment id bbt zelle payment to
24716,880,592,681,59715,28/09/2023 0:00,Uber 09/28 #3168 PMNT RCVD Uber VISA DIRECT WI,0.696,Travel,Uber,uber pmnt rcvd uber visa direct wi
24717,880,592,681,151943,09/08/2023 0:00,Uber 08/09 #3168 PMNT RCVD Uber VISA DIRECT WI,0.830,Travel,Uber,uber pmnt rcvd uber visa direct wi


---

## 4. Using **Qdrant** for threshold-score

In [3]:
import pandas as pd
from utils import clean_text
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

df = pd.read_csv("data.csv")
df = df.dropna(subset=['clnt_id', 'desc'])
df['desc_clean'] = df['desc'].apply(clean_text) 

unique_desc_by_clnt = df.drop_duplicates(subset=['clnt_id', 'desc_clean'])
unique_desc_by_clnt = unique_desc_by_clnt.reset_index(drop=True)
docs = unique_desc_by_clnt['desc_clean'].tolist()
embeddings = model.encode(docs, show_progress_bar=True).astype('float32')


c:\Users\edsol\Desktop\bank-transaction-agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 3232/3232 [10:32<00:00,  5.11it/s]


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

client = QdrantClient(path="qdrant_db")

client.create_collection(
    collection_name="transactions",
    vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE),
)


RuntimeError: Storage folder qdrant_db is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.

In [440]:
points = [
    PointStruct(
        id=i,
        vector=embeddings[i],
        payload={
            "desc_clean": docs[i], 
            "clnt_id": int(row['clnt_id']), 
            "cat": row['cat']
            }
    )
    for i, row in unique_desc_by_clnt.iterrows()
]

client.upsert(
    collection_name="transactions",
    points=points
)

IndexError: index 56515 is out of bounds for axis 0 with size 56515

In [437]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

def search_qdrant(query, client_id, category=None, top_k=100, score_threshold=0.5):
    query_vec = model.encode([clean_text(query)]).tolist()

    conditions = [FieldCondition(key="clnt_id", match=MatchValue(value=client_id))]

    if category:
        conditions.append(FieldCondition(key="cat", match=MatchValue(value=category)))
        
    query_filter = Filter(must=conditions)

    result = client.search(
        collection_name="transactions",
        query_vector=query_vec,
        query_filter=query_filter,
        limit=top_k,
        score_threshold=score_threshold,
        with_payload=True
    )

    return result


In [438]:
results = search_qdrant("how much on cab rides", client_id=880, category="Travel", score_threshold=0.75)
for r in results:
    print(r.payload["desc_clean"], r.score)


C:\Users\edsol\AppData\Local\Temp\ipykernel_31624\653382162.py:21: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = client.search(


ValueError: Multivector  is not found in the collection